In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
def clean(X):
    x = np.copy(X)
    """Remove weird values from the original data set."""
    x[abs(x) ==  999] = np.nan
    mean_x = np.nanmean(x, axis = 0)
    std_x = np.nanstd(x, axis=0)
    rows, cols = x.shape
    for i in range(rows):
        for j in range(cols):
            if(np.isnan(x[i][j])):
                x[i][j] = mean_x[j]
                
    return x, mean_x, std_x

In [3]:
def standardize(x, mean_x, std_x):
    """Standardize values from the cleaned data set."""
    tX = np.copy(x)
    tX = tX - mean_x[np.newaxis, :]
    tX = tX / std_x[np.newaxis, :]
    return tX

In [4]:
def change_y(y):
    y[y == -1.0] = 0
    return y

In [5]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv'
y, X, ids = load_csv_data(DATA_TRAIN_PATH)

In [6]:
x, mean_x, std_x = clean(X)

In [7]:
standarized_x = standardize(x, mean_x, std_x)

In [8]:
yf = change_y(y)

In [9]:
def sample_data(yf, standarized_x, seed, size_samples):
    tX = np.copy(standarized_x)
    y = np.copy(yf)
    
    y = np.expand_dims(y, axis=1)
    np.random.seed(seed)
    num_observations = y.shape[0]
    random_permuted_indices = np.random.permutation(num_observations)
    y = y[random_permuted_indices]
    tX = tX[random_permuted_indices]
    return y[:size_samples,:], tX[:size_samples,:]


In [10]:
def de_standardize(x, mean_x, std_x):
    """Reverse the procedure of standardization."""
    x = x * std_x
    x = x + mean_x
    return x


## Do your thing crazy machine learning thing here :) ...

In [11]:
#====================================================Least-Squares-GD==========================================================


In [12]:
def compute_gradient(y, tx, w):
    """Compute the gradient."""
    err = y - tx.dot(w)
    grad = -tx.T.dot(err) / len(err)
    return grad, err

def calculate_mse(e):
    """Calculate the mse for vector e."""
    return 1 / 2 * np.mean(e ** 2)
def compute_loss(y, tx, w):
    e = y - tx.dot(w)
    return calculate_mse(e)

In [15]:
def least_squares_GD(y, tx,  max_iters, gamma):
    """ Linear regression using gradient descent
    """
    threshold = 1e-8
    # we initialize w to a zeros vector
    w = 0.001* np.ones((tx.shape[1],1))#np.zeros(tx.shape[1])

    # Define parameters to store weight and loss
    loss = 0 
    losses = []
    for n_iter in range(max_iters):
        # compute gradient and loss
        gradient,err = compute_gradient(y, tx, w)
        loss = compute_loss(y, tx, w)
        # store w and loss
        losses.append(loss)
        # update w by gradient
        w = w - gamma * gradient
        # converge criteria
        
    return w, losses


In [16]:
y, tx = sample_data(yf, standarized_x, 23, 100)
max_iters = 10000
gamma = 0.001
lgd_loss, lgd_w =least_squares_GD(y, tx, max_iters, gamma)

In [17]:
#====================================================Least-Squares-SGD==========================================================

In [18]:
def calculate_mse(e):
    
    return 1/2*np.mean(e**2)

In [19]:
def compute_mae_loss(y, tx, w):
   
    e = y - tx.dot(w)
    return calculate_mse(e)

In [20]:
def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]
            


In [21]:
def compute_stoch_gradient(y, tx, w):
    
    error = y - tx.dot(w)
    gradient = -tx.T.dot(error) / len(error)
    return gradient, error

In [22]:
def least_squares_SGD(y, tx, initial_w, max_iters, gamma):
    
    w = initial_w
    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(y, tx, batch_size=1, num_batches=1):
            grad, _ = compute_stoch_gradient(y_batch, tx_batch, w)
            w = w - gamma * grad
            loss = compute_mae_loss(y, tx, w)
    return (w, loss)

In [23]:
y, tX = sample_data(yf, standarized_x, 23, 100)
weights, loss = least_squares_SGD(y, tX, 0.001* np.ones((tX.shape[1],1)), 1000, 0.001)

In [24]:
#==================================================Logistic-Regression==========================================================

In [25]:
def sigmoid(t):
    
    expo = np.exp(-t)
    result = 1.0/(1.0 + expo)
    return result


In [26]:
def compute_gradient(tx, y, w):
    
    pred = sigmoid(tx @ w)
    gradient = tx.T @ (pred - y) 
    return gradient


In [27]:
def compute_sigmoid_loss(tx, y, w):
    
    predictions = sigmoid(tx @ w)
    neg_losses_per_datapoint = -(y * np.log(predictions) + (1 - y) * np.log(1 - predictions))
    return neg_losses_per_datapoint.sum()


In [28]:
def learning_by_gradient_descent(y, tx, w, gamma):
    """
    Do one step of gradient descent using logistic regression.
    Return the loss and the updated w.
    """
    # ***************************************************
    loss = compute_sigmoid_loss(tx, y, w)
    # ***************************************************
    gradient = compute_gradient(tx, y, w)
    # ***************************************************
    w = w - gamma * gradient
    # ***************************************************
    return loss, w



In [29]:
def logistic_regression(y, tx, initial_w, max_iters, gamma):
    
    # init parameters
    threshold = 1e-8
    losses = []
    w = np.concatenate(([[1]], np.copy(initial_w)), axis=0)
    # build tx
    tX = np.c_[np.ones((y.shape[0], 1)), tx]
    # start the logistic regression
    for iter in range(max_iters):
        # get loss and update w.
        loss, w = learning_by_gradient_descent(y, tX, w, gamma)
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            return (w, loss)
    return (w, loss)


In [30]:
y, tX = sample_data(yf, standarized_x, 23, 100)
weights, loss = logistic_regression(y, tX, 0.001* np.ones((tX.shape[1],1)), 1000, 0.001)

In [31]:
#====================================================Regularized Logistic Regression==========================================================

In [32]:
def sigmoid(t):
    """apply sigmoid function on t."""
    return 1.0 / (1 + np.exp(-t))

def calculate_gradient(y, tx, w):
    """compute the gradient of loss."""
    pred = sigmoid(tx.dot(w))
    grad = tx.T.dot(pred - y)
    return grad
def calculate_loss(y, tx, w):
    """compute the cost by negative log likelihood."""
    pred = sigmoid(tx.dot(w))
    loss = y.T.dot(np.log(pred)) + (1 - y).T.dot(np.log(1 - pred))
    return np.squeeze(- loss)

def penalized_logistic_regression(y, tx, w, lambda_):
    """return the loss and gradient."""
    num_samples = y.shape[0]
    loss = calculate_loss(y, tx, w) + lambda_ * np.squeeze(w.T.dot(w))
    gradient = calculate_gradient(y, tx, w) + 2 * lambda_ * w
    return loss, gradient

def learning_by_penalized_gradient(y, tx, w, gamma, lambda_):
    """
    Do one step of gradient descent, using the penalized logistic regression.
    Return the loss and updated w.
    """
    loss, gradient = penalized_logistic_regression(y, tx, w, lambda_)
    w -= gamma * gradient
    return loss, w

In [33]:
def reg_logistic_regression(y, tx, lambda_, max_iters, gamma):
    """Regularized logistic regression"""
   # we initialize it to a zeros vector
    w = 0.001* np.ones((tX.shape[1],1))#np.zeros(tx.shape[1])
    
    losses = []
    threshold = 0.1 # 1e-8

    # start the logistic regression
    for iter in range(max_iters):
        # get loss and update w.
        w, loss = learning_by_gradient_descent(y, tx, w, gamma)
        losses.append(loss)

        # converge criteria
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break

    norm = sum(w ** 2)
    cost = w + lambda_ * norm / (2 * np.shape(w)[0])

    return w, cost


In [34]:
y, tX = sample_data(yf, standarized_x, 23, 100)
weights, loss = reg_logistic_regression(y, tX, 0.001* np.ones((tX.shape[1],1)), 1000, 0.001)

ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

## Generate predictions and save ouput in csv format for submission:

In [92]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
tX_test, mean, std = clean(tX_test)
tX_test = standardize(tX_test, mean, std)

In [93]:
OUTPUT_PATH = '../data/sample-submission.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(np.squeeze(weights[1:,:]), tX_test)
print(y_pred)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

[ 1. -1. -1. ...  1.  1. -1.]
